In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import pickle
import orchest
import json

data = orchest.get_inputs() # data = [(response)]
response = json.loads(data["response"])
outliers_fraction = orchest.get_step_param("outliers_fraction")
print("outliers fraction is set to: ", outliers_fraction)

In [ ]:
dates = response["result"]["data"]["times"]
values = list(response["result"]["data"]["series"].values())[0]
df = pd.DataFrame({'date': dates, 'x': values})

In [ ]:
scaler = StandardScaler()
np_scaled = scaler.fit_transform(df[["x"]])
data = pd.DataFrame(np_scaled)

In [ ]:
random_state = np.random.RandomState(4)
model =  IsolationForest(contamination=outliers_fraction, random_state=random_state)
model.fit(data) 
print(model.get_params())

In [ ]:
df['anomaly_score'] = pd.Series(model.predict(data))
anomaly = df.loc[df['anomaly_score'] == -1, ['date', 'x']] 

print("anomalies found:")
anomaly

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x = df['date'], y = df['x'], mode='markers', marker=dict(color='blue', size=4)))
fig.add_traces(go.Scatter(x = anomaly['date'], y = anomaly['x'], textposition='top left',
                          textfont=dict(color='#233a77'),
                          mode='markers+text',
                          marker=dict(color='red', size=6)))

fig.show()

In [ ]:
file = '../data/model.sav'
f = open(file,'x')
f.close()

with open(file, 'wb') as f:
    print("Save model in model file...")
    pickle.dump(model, f)
print("Done!")